In [1]:
import numpy as np 
import pandas as pd 
import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

print(os.listdir("../input"))

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

Using TensorFlow backend.


['Hindi_English_Truncated_Corpus.csv']


In [2]:
lines=pd.read_csv("../input/Hindi_English_Truncated_Corpus.csv",encoding='utf-8')

In [3]:
lines.head()

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what needs to be done.,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह करने कि अनुमति नहीं है ."
1,ted,"I'd like to tell you about one such child,","मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहूंगी,"
2,indic2012,This percentage is even greater than the percentage in India.,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not paying attention.,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called Upanishad.,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।


In [4]:
lines['source'].value_counts()

tides        50000
ted          39881
indic2012    37726
Name: source, dtype: int64

In [5]:
lines=lines[lines['source']=='ted']

In [6]:
lines.head()

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what needs to be done.,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह करने कि अनुमति नहीं है ."
1,ted,"I'd like to tell you about one such child,","मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहूंगी,"
3,ted,what we really mean is that they're bad at not paying attention.,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
7,ted,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
13,ted,So there is some sort of justice,तो वहाँ न्याय है


In [7]:
pd.isnull(lines).sum()

source              0
english_sentence    0
hindi_sentence      0
dtype: int64

In [8]:
lines=lines[~pd.isnull(lines['hindi_sentence'])]

In [9]:
lines.drop_duplicates(inplace=True)

In [10]:
lines.shape

(38803, 3)

* ### Let us pick any 30000 rows from the dataset.

In [11]:
lines=lines.sample(n=30000,random_state=42)
lines.shape

(30000, 3)

In [12]:
# Lowercase all characters
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.lower())
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.lower())

In [13]:
# Remove quotes
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub("'", '', x))
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub("'", '', x))

In [14]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['english_sentence']=lines['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [15]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.translate(remove_digits))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.translate(remove_digits))

lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.strip())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.strip())
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))


In [16]:
# Add start and end tokens to target sequences
lines['english_sentence'] = lines['english_sentence'].apply(lambda x : 'START_ '+ x + ' _END')

In [17]:
lines.head()

,source,english_sentence,hindi_sentence
82040,ted,START_ we still dont know who her parents are who she is _END,हम अभी तक नहीं जानते हैं कि उसके मातापिता कौन हैं वह कौन है
85038,ted,START_ no keyboard _END,कोई कुंजीपटल नहीं
58018,ted,START_ but as far as being a performer _END,लेकिन एक कलाकार होने के साथ
74470,ted,START_ and this particular balloon _END,और यह खास गुब्बारा
122330,ted,START_ and its not as hard as you think integrate climate solutions into all of your innovations _END,और जितना आपको लगता है यह उतना कठिन नहीं हैअपने सभी नवाचारों में जलवायु समाधान को एकीकृत करें


In [18]:
### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in lines['english_sentence']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in lines['hindi_sentence']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [19]:
len(all_eng_words)

15350

In [20]:
len(all_hindi_words)

19404

In [21]:
lines['length_eng_sentence']=lines['english_sentence'].apply(lambda x:len(x.split(" ")))
lines['length_hin_sentence']=lines['hindi_sentence'].apply(lambda x:len(x.split(" ")))

In [22]:
lines.head()

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
82040,ted,START_ we still dont know who her parents are who she is _END,हम अभी तक नहीं जानते हैं कि उसके मातापिता कौन हैं वह कौन है,13,14
85038,ted,START_ no keyboard _END,कोई कुंजीपटल नहीं,4,3
58018,ted,START_ but as far as being a performer _END,लेकिन एक कलाकार होने के साथ,9,6
74470,ted,START_ and this particular balloon _END,और यह खास गुब्बारा,6,4
122330,ted,START_ and its not as hard as you think integrate climate solutions into all of your innovations _END,और जितना आपको लगता है यह उतना कठिन नहीं हैअपने सभी नवाचारों में जलवायु समाधान को एकीकृत करें,18,18


In [23]:
lines[lines['length_hin_sentence']>30].shape

(0, 5)

In [24]:
lines=lines[lines['length_eng_sentence']<=20]
lines=lines[lines['length_hin_sentence']<=20]

In [25]:
lines.shape

(29893, 5)

In [26]:
print("maximum length of Hindi Sentence ",max(lines['length_hin_sentence']))
print("maximum length of English Sentence ",max(lines['length_eng_sentence']))

maximum length of Hindi Sentence  20
maximum length of English Sentence  20


In [27]:
max_length_src=max(lines['length_hin_sentence'])
max_length_tar=max(lines['length_eng_sentence'])

In [28]:
input_words = sorted(list(all_hindi_words))
target_words = sorted(list(all_eng_words))
num_encoder_tokens = len(all_hindi_words)
num_decoder_tokens = len(all_eng_words)
num_encoder_tokens, num_decoder_tokens

(19404, 15350)

In [29]:
num_decoder_tokens += 1 #for zero padding


In [30]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [31]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [32]:
lines = shuffle(lines)
lines.head(10)

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
99782,ted,START_ my husband left town with the kids _END,मेरे पति बच्चों को लेकर शहर छोड़ कर चले गए,9,10
65041,ted,START_ and they figure it out _END,और यह कर दिखाने में वे सफल हो जाते हैं,7,10
100354,ted,START_ so its actually a multidimensional user interface _END,असल में यह एक बहु आयामी संकेतक है,9,8
91533,ted,START_ about the image that is constructed about us _END,और उस अक्स का जो हमारे आसपास बना दिया गया है,10,11
114808,ted,START_ id been conducting for years and i suddenly had a realization _END,मैं साल से संगीत निर्देशक था और अचानक मुझे एक अनुभूति हुई,13,12
48308,ted,START_ even before she saw me _END,उसे मुझपर तरस आने लगा था,7,6
59874,ted,START_ the boundaries between nations mean less and less _END,जिसमें देशों के बीच की सीमाओं का अर्थ क्षीण होते जा रहा है,10,13
119450,ted,START_ and my great big internet trunk line _END,और मेरी विशाल इंटरनेट ट्रंक लाइन,9,6
94304,ted,START_ somewhere in here there are actually _END,और यहाँ कहीं वास्तव में,8,5
104282,ted,START_ and ive looked at the crossborder component _END,मैं सीमा पार से घटक में देखा है,9,8


### Split the data into train and test

In [33]:
X, y = lines['hindi_sentence'], lines['english_sentence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((23914,), (5979,))

### Let us save this data

In [34]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [35]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder-Decoder Architecture

In [36]:
latent_dim=300

In [37]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

Instructions for updating:
Colocations handled automatically by placer.


In [38]:
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [39]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [40]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    5821200     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 300)    4605300     input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LS

In [41]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 100

In [42]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/100
186/186 [==============================] - 83s 449ms/step - loss: 6.1397 - val_loss: 5.8552
Epoch 2/100
186/186 [==============================] - 77s 416ms/step - loss: 5.5642 - val_loss: 5.5200
Epoch 3/100
186/186 [==============================] - 77s 416ms/step - loss: 5.2259 - val_loss: 5.3561
Epoch 4/100
186/186 [==============================] - 78s 418ms/step - loss: 4.9824 - val_loss: 5.2401
Epoch 5/100
186/186 [==============================] - 78s 418ms/step - loss: 4.7672 - val_loss: 5.1732
Epoch 6/100
186/186 [==============================] - 78s 417ms/step - loss: 4.5778 - val_loss: 5.0586
Epoch 7/100
186/186 [==============================] - 78s 418ms/step - loss: 4.4028 - val_loss: 5.0384
Epoch 8/100
186/186 [==============================] - 78s 420ms/step - loss: 4.2412 - val_loss: 5.0168
Epoch 9/100
186/186 [=====================

In [43]:
model.save_weights('nmt_weights.h5')

In [45]:
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [46]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = target_token_index['START_']
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        states_value = [h, c]

    return decoded_sentence

In [48]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1


In [49]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Hindi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

Input Hindi sentence: ♫ यह एक अनुमान है ♫
Actual English Translation:  ♫ thats a guess ♫ 
Predicted English Translation:  ♫ thats a guess ♫ 


In [50]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Hindi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

Input Hindi sentence: पिछले वर्ष विभिन्न कारणों से
Actual English Translation:  last year for various reasons 
Predicted English Translation:  last year for various reasons 


In [51]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Hindi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

Input Hindi sentence: ऐसे व्यक्ति के उदाहरण के रूप में लिया
Actual English Translation:  as an example of a gentleman 
Predicted English Translation:  as an example of a gentleman 


In [53]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Hindi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

Input Hindi sentence: इसका अर्थ है कि भीतर से आप जैसे हैं उसमें ख़ुशी पायें
Actual English Translation:  its just about being you and being cool with that 
Predicted English Translation:  its just about the way you asked it got to eigh


In [52]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Hindi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

Input Hindi sentence: यहाँ बच्चों ने अपना खुद का संगीत रिकॉर्ड किया
Actual English Translation:  where the children recorded their own music 
Predicted English Translation:  where the children have taken cricket to go 
